In [1]:
#meta 5/7/2024 Try Gemma 
#Goal: Code snippets w/ Gemma foundation model
# started from https://huggingface.co/google/gemma-2b (weird), switched to https://huggingface.co/google/gemma-2b-it


#infra: Trainbox + VSCode 
#      env: tc2-venv created by Blake
#      confirmed Python 3.10.4
#      numpy 1.24.2, pandas 2.0.0, added scikit-learn 1.2.2, req scipy
#      [not any more pip 22.0.4], ipykernel 6.22.0, ipython 8.12.0
#
#for HuggingFace added
#      tqdm-4.66.2 huggingface-hub-0.21.1 [safetensors-0.4.2 tokenizers-0.15.2] transformers-4.38.1
#need a lower version `ulrlib3` -> downgrade it with $ pip install requests==2.27.1 -> urllib3 1.26.18
#needs pip 24.0
#for HuggingFace Sentence Embeddings added
#      sentence-transformers-2.5.1


#input: n/a 
#output: n/a


#history
#5/7/2024 TRY HF GEMMA 
#      2B instruct version of the Gemma model "google/gemma-2b-it"

#$config $private

#References
#HF Gemma
# https://huggingface.co/google/gemma-2b-it

#Getting Started with Google’s Gemma LLM using HuggingFace Libraries  
# https://medium.com/@coldstart_coder/getting-started-with-googles-gemma-llm-using-huggingface-libraries-a0d826c552ae#


import os 
import sys
import time as time
import numpy as np

#ML
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [4]:
#python version $my
print(sys.version)
print(np.__version__)

3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]
1.24.2


In [5]:
#----- GlOBAL VARS -----
###hf_token = #"get your token in http://hf.co/settings/tokens" $private

# identify which checkpoint we want, this is the repository on huggingface that we'll pull the model from
MODEL_CHECKPOINT = "google/gemma-2b-it"

In [6]:
#$config network settings

#set proxy needed to fix SSL error
os.environ['REQUESTS_CA_BUNDLE'] = '../../myCreds/all_ca_certs.crt'
# #os.environ['CURL_CA_BUNDLE'] = '../../myCreds/all_ca_certs.pem'

proxy = 'http://url:8080' #$config
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

# Try Gemma
Using HF model https://huggingface.co/google/gemma-2b-it

## Step 2. Model   

### Load a pre-trained LM 

How to: Access gated repo issue!  
Refer to https://huggingface.co/google/gemma-2b/discussions/11

In [7]:
# load in the tokenizer,
# just in case you aren't familiar with llms the tokenizer takes the raw text and converts it into a vector that can be processed by our model,
# so basically it's a converter to convert english into numbers for our model to compute on
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(MODEL_CHECKPOINT, token=hf_token)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt") 

outputs = model.generate(**input_ids, max_length = 100)
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


<bos>Write me a poem about Machine Learning.

Machines, they weave and they learn,
From the data, they discern.
Algorithms, a symphony,
Unleash the power of the machine.

With each iteration, they grow,
A tapestry of insights they sow.
From the past, they learn and they adapt,
A never-ending, ever-fast.

The future holds mysteries untold,
Where machines will shape our world.
They will heal, they


In [8]:
#multi input
input_text = [
    "I love sushi!",
    "I've been waiting for this blog my whole life.",
]
input_ids = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt") 

outputs = model.generate(**input_ids, max_length = 50)
print(tokenizer.decode(outputs[0]))
print(tokenizer.decode(outputs[1]))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<pad><pad><pad><pad><pad><pad><pad><pad><bos>I love sushi! But I'm not a seasoned sushi chef, and I'm not sure where to start.

**What's the best way to learn to make sushi at home?**
<bos>I've been waiting for this blog my whole life. It's finally here, and I'm so excited to read it.

I've been searching for a blog that truly understands the struggles and joys of life, and I


### Additional Gen Parameters
Example: Getting Started with Google’s Gemma LLM using HuggingFace Libraries  
https://medium.com/@coldstart_coder/getting-started-with-googles-gemma-llm-using-huggingface-libraries-a0d826c552ae

In [13]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt") 

token_outputs = model.generate(**input_ids, max_new_tokens=150, do_sample=True, temperature=0.5)
# since this is exploratory we'll decode special tokens as well. for this prompt
# it will be the beginning <bos> token and the ending <eos> tokens.

print(tokenizer.decode(token_outputs[0], skip_special_tokens=False))


<bos>Write me a poem about Machine Learning.

Machines, with gears and wires,
Unleash a power that's beyond compare.
Algorithms dance, a symphony,
Learning from data, setting a new free.

From medical scans to social media's sway,
They shape our world, come what may.
With every query, a new insight unfolds,
A world of possibilities, a story to be told.

But be warned, the path is not without blight,
Bias and discrimination, a dark night.
Ethical concerns, a constant fight,
To ensure fairness, a beacon of light.

So let us embrace this wondrous age,
Where machines and humans, hand in hand, engage.
With every step, we learn and grow,


In [9]:
mystop

NameError: name 'mystop' is not defined

# Xtra

In [ ]:
#$xtra would it work as a sentence transformer?
# not Sentence Embeddings
from sentence_transformers import SentenceTransformer
#from sentence_transformers import util

t0 = time.time()

hf_model = SentenceTransformer('google/gemma-2b') #google-bert/bert-base-uncased') #"all-MiniLM-L12-v2") #"wvprevue/e5-mistral-7b-instruct") #all-MiniLM-L6-v2

print ("\LM load time: in {:.8f}(min): ".format( (time.time() - t0)/60 ))


No sentence-transformers model found with name google/gemma-2b. Creating a new one with MEAN pooling.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.96s/it]


\LM load time: in 0.26163914(min): 
